# Hướng Dẫn Sử Dụng FastAPI Trên Google Colab Với Ngrok

## Mục Lục
- [Bước 1: Lấy ngrok Token](#bước-1-lấy-ngrok-token)
- [Bước 2: Cài đặt môi trường](#bước-2-cài-đặt-môi-trường)

---

## Bước 1: Lấy ngrok Token
1. Truy cập [ngrok.com](https://ngrok.com/) và đăng ký tài khoản nếu bạn chưa có.
2. Sau khi đăng nhập, truy cập vào **Dashboard**.
3. Sao chép **ngrok auth token** từ mục **Get Started** hoặc **Your Authtoken**.

---

## Bước 2: Cài đặt môi trường
1. Mở Google Colab và tạo một notebook mới.
2. Chạy lệnh sau để cài đặt các thư viện cần thiết:
   ```python
   !pip install fastapi nest_asyncio pyngrok pydantic transformers torch uvicorn


In [ ]:
!pip install -q fastapi uvicorn transformers torch
!pip install -q ngrok

In [ ]:
from fastapi import FastAPI, HTTPException, Body, Request
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModel
import torch
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn

# Khai báo thư viện userdata để import token bí mật
from google.colab import userdata
conf.get_default().auth_token = userdata.get('ngrok_token')

# Khởi động ngrok
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

# Khai báo GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Bắt buộc dùng cho Colab
nest_asyncio.apply()

# Tạo ứng dụng FastAPI
app = FastAPI()

# Load model và tokenizer
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModel.from_pretrained("VietAI/vit5-base").to(device)

# Định nghĩa input schema
class TextListInput(BaseModel):
    texts: list[str]

@app.post("/get_cls")
async def get_cls(request: Request, input: TextListInput = Body(...)):
    try:
        # Tokenize danh sách văn bản
        inputs = tokenizer(
            input.texts, 
            return_tensors="pt", 
            truncation=True, 
            padding="max_length", 
            max_length=512
        ).to(device)

        # Lấy output từ mô hình
        with torch.no_grad():
            outputs = model.encoder(**inputs)
            cls_tokens = outputs.last_hidden_state  # Lấy token CLS cho mỗi văn bản
        cls_tokens = cls_tokens[:, 0, :].cpu().numpy().tolist() 
        print(cls_tokens)

        # Trả về danh sách CLS token
        return {"cls_tokens": cls_tokens}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



# # Khởi chạy ứng dụng
uvicorn.run(app, host="0.0.0.0", port=5000)
